In [1]:

"""
Dependencies for the network :
    =====> neurolab
    =====> matplotlib.pylab 
    =====> numpy 
Author : Ayodeji 
Date : 15-08-2018
Project Objective:
    This neural network model objective is to predict the future unemployment rate in Nigeria using neural Networks.
"""

import numpy as np
import neurolab as nln
from sklearn.metrics  import accuracy_score
import matplotlib.pylab as plt

class MultilayeredANNet:
    
    def __init__(self , 
                 input_data , # This is the input data we will use to train our network and make predictions
                 input_data_len ,
                 out_data ,
                 w1 = None ,
                 w2 = None,
                 verbose = False, # displays verbose messages 
                 debug = False,
                 logging = False,
                 log_file = "",
                 epochs = None,  # time to train the network 
                 net_learning_rate = None # this is the learning rate of the neural network 
                ):
        """
           Example : [2 ,34 ,43, 3] ----> input_data_len  = 4
           
           epochs : This is the number of times the machine will be trained to understand the patterns behind the datasets given to it
           w1: This parameter is a string object that holds the FIRST weight of the network
           w2: This parameter is a string object that holds the SECOND weight of the network
        """
        
        # save the errors of the trained network
        self.error_deltas = np.array([]) ; 
        
        self.trained = False ; #this is set to false so that the machined is trained before calling predict() function
        
        self.input_data = X/np.amax(input_data, axis=0) # maximum of X array
        self.input_data_len = input_data_len
        
        self.out_data = out_data / 100 ; # this is the output of the given input to help the machine train itself
        self.output_data_len = 1 ; # this is the node length(size) of the output layer
        
        self.hidden_layer_node_len = 3 ;# the number of neuron the hidden layer will hold  
        self.network_layers = 3
                
        self.biases = np.random.randint(0 , dtype=int , size=2 , high=2); #this are the biases that will addes to the Ann model
       
    
        #down here we scale the data according to the the max value present in each array
        
        
        #when the machine does not have a trained weight 
        #so a random new weight will be generated here
        if w1 == None and w2 == None :
            #since we have three layers for the neural network, then we are going to build the network with two weights arrays
            #of hiddenlayer_node_lenght * input_data_len matrix 
            self.W1 = np.random.randn(self.input_data_len , self.hidden_layer_node_len ) #  input_data_len(row) * 15(column) 
            self.W2 = np.random.randn(self.hidden_layer_node_len , self.output_data_len) # 15(row) * output_data_len
        
        #this will run when the machine has a trained dataset it wants to use 
        else :
            self.trained = True ; #this is automatically trained when the weight is supplied to the network
            self.W1 = np.loadtxt(w1) # will load the first weight and assign it to W1
            self.W2 = np.loadtxt(w2) # will load the second weight and assign it to w2
        
    def saveWeights(self):
        np.savetxt("w1.txt", self.W1, fmt="%s")
        np.savetxt("w2.txt", self.W2, fmt="%s")
    
    
    #this below function is the activation function which is used for the switching 
    def activation_func(self , s):
        """
        This function is used to trigger the switching when the ouput is greater than a certain threshold it
        changes the value of the ouput 1, when it is lesser it value between ( 0 --> 1 or -1 ---> 1)
        """
        return 1 / (1 + np.exp(-s))   
    
    def sigmoidPrime(self , s ):
        #s----> is a numpy array object 
        return s * ( 1 - s)
    
    def matrix_feed_forward_nn(self , node_in):
            self.z1 = self.input_data.dot(self.W1)# + self.biases[0]
            self.z2 = self.activation_func(self.z1)  #finds the dot products of the weight and the input node and add the biases
            self.z3 = self.z2.dot(self.W2) #+ self.biases[1];
            
            output = self.activation_func(self.z3)
            #print("z========> " , self.z3)
            #print("Activation Function==>" , self.activation_func(output))
            
            return output # returns the ouput of the result
    
    #This tain how the machine to be able to get the correct set of weight to predict any data giving to it
    #The biases of the neural network is also trained. The biases is also known as THRESHOLDS
    def __train_machine__(self , input_data ,out_data):
        o = self.matrix_feed_forward_nn(input_data)
        self.backpropagation(input_data , out_data , o) # this part will update the weight
        
    def predict(self , data_to_predcit):
        #if the machine is not trained, no prediction will be performed on the data provided
        if self.trained == False :
            print("[ TRAIN_ERROR ] : The network is not trained...This network can not predict the result of the input data.\n"
            +"please train the network first and then call this function again.")
            return
        else:
            return self.matrix_feed_forward_nn(data_to_predcit)
        
    #This function will update the weight of the network so that the machine to learn and predict
    def backpropagation(self , input_data , out_data  ,node_out_results):
        
        self.o_error = out_data - node_out_results # finds the error 
        
        self.error_deltas.append(self.o_error) # appends all the output layer data
        
        self.o_delta = self.o_error * (self.sigmoidPrime(node_out_results))
        
        #the hidden layer(s) size is 1
        self.z2_error = self.o_delta.dot(self.W2.T)
        
        self.z2_delta = self.z2_error * (self.sigmoidPrime(self.z2))
        
        #This following statements will update the weights (synapses) of the network
        self.W1 +=input_data.T.dot(self.z2_delta)# adjusting first set (hidden --> output) weights
        
        self.W2 +=self.z2.T.dot(self.o_delta)# adjusting second set (input --> hidden) weights
        
        #updating the biases of the network from here
        #self.biases[0] += self.self.input_data.T.dot(z2_delta)
        #self.biases[1] += self.z2.T.dot(o_delta)
       
    def train(self):
        for i in range(1000):
          #print("Initial Weight======> \nW1 : %s\nW2 : %s" % (self.W1 , self.W2) )
          #print("# " + str(i) + "\n")
          #print ("Input (scaled): \n" + str(self.input_data) )
          #print ("Actual Output: \n" + str(self.out_data) )
          #print ("Loss: \n" + str(np.mean(np.square(self.out_data - self.matrix_feed_forward_nn(self.input_data)))) ) # mean sum squared loss
          #print ("\n")
          self.__train_machine__(self.input_data , self.out_data)
        
        # this is to indicate that the machine is beign trained
        print ("Predicted Output: \n" + str(self.matrix_feed_forward_nn(self.input_data)) )
        self.trained = True ;
    #this network train the data using Neurolab package 
    #It trains the network by supplying input and ouput data to the network
    #The input data is the training set and while the ouput data is the target label
    def train_nln(self, show_error_graph = None):
        try :
            self.nln_error_data = nln.train(input_data ,out_data) 
            #-------------------------------------TRAINIGN THE NETWORK MODEL---------------------------------------------------------
            if show_error_graph:
                plt.plto(self.nln_error_data) # plot the np array error data from the taining function model
                plot.grid() # Shows a grid view when the errro_data is plotted
                #after the training is done the model will save the weights of the neural network
                nln.save("trained_unemployment_weights.txt") # the trained weights are saved into the txt file
            #---------------------------------ERROR HANDLING-----------------------------------------------
            
            def get_weight_and_biases():
                self.weights = np.array([[]]); # @2Dim array object will be creted for the network weights
                self.biases  = np.array([]); # 1Dim array object that will hold the biases of the neural network
                self.wb_holder = nln.load("trained_unemployment_weights.txt") #loads the neural network weights and biases
                
                #This iteration pushes the weights of the trained network and biases into the 
                # arrays created above ^
                for layer_index in range(0, len(self.wb_holder.layers)):
                    self.weights.append(self.wb_holder.layers[layer_index].np['w']) # appends the trained weights
                    self.biases.append(self.wb_holder.layers[layer_index].np['b'])# appends the trained biases
                    
                
        except Exception:
            print("There is an argument ERROR_.")
        
        

X = np.array(([2, 9], [1, 5], [3, 6]), dtype=float)
y = np.array(([92], [86], [89]), dtype=float)
        
Ann = MultilayeredANNet(input_data= X , input_data_len= len(X[0]) , out_data= y)
Ann.train()




AttributeError: 'numpy.ndarray' object has no attribute 'append'